<a href="https://colab.research.google.com/github/Danny-Dasilva/Super-Resolution/blob/main/Making_Slowmo_RIFE_Simplified(youtube).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RIFE: Real-Time Intermediate Flow Estimation for Video Frame Interpolation

This notebook is based on (https://rife-vfi.github.io/)

In [ ]:
#@title STEP1: Setup

#@markdown * Installs hzwer/arXiv2020-RIFE
#@markdown * Download pre-trained Model


!git clone https://github.com/hzwer/arXiv2020-RIFE
!pip install -q youtube-dl
#download the model from google drive
def download_from_google_drive(file_ids, file_name):
  for id in file_ids:
    print("Attept download from", id)
    response = !gdown --id $id --output $file_name
    if response[0] == 'Downloading...':
      break
    else:
      continue

file_ids = ["1wsQIhHZ3Eg4_AfCXItFKqqyDMB4NS0Yd"]

download_from_google_drive(file_ids, "RIFE_trained_model_HDv2.zip")
!7z e RIFE_trained_model_HDv2.zip


!mkdir /content/arXiv2020-RIFE/train_log
!mv *.pkl /content/arXiv2020-RIFE/train_log/
%cd /content/arXiv2020-RIFE/
!pip3 install -r requirements.txt


def show_local_mp4_video(file_name, slowmoFactor, width=640, height=480):
  import io
  import base64
  from IPython.display import HTML
  video_encoded = base64.b64encode(io.open(file_name, 'rb').read())
  return HTML(data='''
                        <script>
                          var vid = document.getElementById("myVideo");
                          vid.playbackRate = 1/{3};
                        </script> 
                        <video width="{0}" height="{1}" alt="test" id="myVideo" controls>
                        <source src="data:video/mp4;base64,{2}" type="video/mp4" />
                      </video>'''.format(width, height, video_encoded.decode('ascii'), slowmoFactor))


In [ ]:
# @title STEP2: Apply on a Youtube Video
YOUTUBE_URL = 'https://www.youtube.com/watch?v=QJO3ROT-A4E' #@param {type:"string"}
from IPython.display import clear_output
#find youtube video id
from urllib import parse as urlparse
url_data = urlparse.urlparse(YOUTUBE_URL)
query = urlparse.parse_qs(url_data.query)
YOUTUBE_ID = query["v"][0]

#displays your youtube video
# YouTubeVideo(YOUTUBE_ID)

SLOW_MOTION_FACTOR =  '8' #@param ["2", "4", "8", "16"]
#@markdown ---
#@markdown <small>Making slow mo of entire video can take long time so ..</small>

#@markdown ### Trim the video (start, end) seconds
start = 51 #@param {type:"integer"}

end =  53#@param {type:"integer"}


#@markdown ---

interval = end - start

!rm -df youtube.mp4
# download the youtube with the given ID
!youtube-dl -f 'bestvideo[ext=mp4]' --output "youtube.%(ext)s" https://www.youtube.com/watch?v=$YOUTUBE_ID

#find the frame_rate of driving video
from fractions import Fraction
frame_rate_source = !ffprobe -v error -select_streams v -of default=noprint_wrappers=1:nokey=1 -show_entries stream=r_frame_rate youtube.mp4

frame_rate_source = int(Fraction(frame_rate_source[0]))
print("FPS=", frame_rate_source)

TARGET_FPS = frame_rate_source * int(SLOW_MOTION_FACTOR)

#delete video.mp4 if already exits
!rm -f video.mp4

# cut the video
# !ffmpeg -y -loglevel info -i youtube.mp4 -t 5 video.mp4
!ffmpeg -y -i youtube.mp4 -ss {start} -t {interval} -async 1 video.mp4


#delete if file already exists
!rm -f output.mp4
clear_output()
%cd /content/arXiv2020-RIFE/
import math
SM_EXP = int(math.log2(int(SLOW_MOTION_FACTOR)))
!python3 inference_video.py --exp=$SM_EXP --video=video.mp4 --output=output.mp4 --skip





#find the frame_rate of source video
from fractions import Fraction
frame_rate_source = !ffprobe -v error -select_streams v -of default=noprint_wrappers=1:nokey=1 -show_entries stream=r_frame_rate video.mp4
frame_rate_source = float(Fraction(frame_rate_source[0]))

#if fps > 30 overwrite frame_rate_source = 30
if frame_rate_source > 30:
  frame_rate_source = 30

#slow down both video
!rm -f video_slow.mp4
!rm -f output_slow.mp4
!ffmpeg  -r {frame_rate_source/int(SLOW_MOTION_FACTOR)} -i video.mp4 -y video_slow.mp4
!ffmpeg -r {frame_rate_source} -i output.mp4 -y output_slow.mp4

#combile two videos
!rm -f output_4_display.mp4
!ffmpeg \
  -i output_slow.mp4 \
  -i video_slow.mp4 \
  -filter_complex '[0:v]pad=iw*2:ih[int];[int][1:v]overlay=W/2:0[vid]' \
  -map [vid] \
  -c:v libx264 \
  -crf 23 \
  -preset veryfast \
  -y output_4_display.mp4
clear_output()
show_local_mp4_video('output_4_display.mp4', width=900, height=253, slowmoFactor = 1)



In [ ]:
#@title STEP3: Download the Ouput
from google.colab import files
file = 'output_slow.mp4' #@param ["output.mp4", "output_slow.mp4", "output_4_display.mp4"]
filepath = {"output.mp4": "/content/arXiv2020-RIFE/output.mp4",
            "output_slow.mp4": "/content/arXiv2020-RIFE/output_slow.mp4", 
            "output_4_display.mp4": "/content/arXiv2020-RIFE/output_4_display.mp4"}
files.download(filepath[file])

In [ ]:
#@title STEP4: Apply on a Your Video
#@markdown <small>Upload your video first then copy its path</small>
PATH_TO_YOU_VIDEO = '/content/video (2).mp4' #@param {type:"string"}
from IPython.display import clear_output
def path_leaf(path):
  import ntpath
  head, tail = ntpath.split(path)
  return tail or ntpath.basename(head)

VIDEONAME = path_leaf(PATH_TO_YOU_VIDEO)

VIDEONAME = '\"' + VIDEONAME +'\"'
VIDEOPATH = '\"' + PATH_TO_YOU_VIDEO +'\"'

SLOW_MOTION_FACTOR =  '4' #@param ["2", "4", "8", "16"]
# TARGET_FPS = 240 #@param {type:"integer"}
#@markdown ---
#@markdown <small>Making slow mo of entire video can take long time so ..</small>

#@markdown ### Trim the video (start, end) seconds
start =  0#@param {type:"integer"}

end =  1#@param {type:"integer"}


#@markdown ---

interval = end - start


#find the frame_rate of driving video
from fractions import Fraction
frame_rate_source = !ffprobe -v error -select_streams v -of default=noprint_wrappers=1:nokey=1 -show_entries stream=r_frame_rate $VIDEOPATH
frame_rate_source = int(Fraction(frame_rate_source[0]))
print("FPS=", frame_rate_source)

TARGET_FPS = frame_rate_source * int(SLOW_MOTION_FACTOR)

#delete if file already exists
!rm -f myCutvideo.mp4
# cut the video
# !ffmpeg -y -loglevel info -i youtube.mp4 -t 5 video.mp4
!ffmpeg -i {VIDEOPATH} -ss {start} -t {interval} -async 1 myCutvideo.mp4

#delete if file already exists
!rm -f myoutput.mp4


clear_output()
%cd /content/arXiv2020-RIFE/
import math
SM_EXP = int(math.log2(int(SLOW_MOTION_FACTOR)))
!python3 inference_video.py --exp=$SM_EXP --video=myCutvideo.mp4 --output=myoutput.mp4 --skip

#find the frame_rate of source video
from fractions import Fraction
frame_rate_source = !ffprobe -v error -select_streams v -of default=noprint_wrappers=1:nokey=1 -show_entries stream=r_frame_rate myCutvideo.mp4
frame_rate_source = float(Fraction(frame_rate_source[0]))

#if fps > 30 overwrite frame_rate_source = 30
if frame_rate_source > 30:
  frame_rate_source = 30


#slow down both video
!rm -f myvideo_slow.mp4
!rm -f myoutput_slow.mp4
!ffmpeg -r {frame_rate_source/int(SLOW_MOTION_FACTOR)} -i myCutvideo.mp4 -y myvideo_slow.mp4
!ffmpeg -r {frame_rate_source} -i myoutput.mp4 -y myoutput_slow.mp4

#combile two videos
!ffmpeg \
  -i myoutput_slow.mp4 \
  -i myvideo_slow.mp4 \
  -filter_complex '[0:v]pad=iw*2:ih[int];[int][1:v]overlay=W/2:0[vid]' \
  -map [vid] \
  -c:v libx264 \
  -crf 23 \
  -preset veryfast \
  -y myoutput_4_display.mp4
clear_output()
show_local_mp4_video('myoutput_4_display.mp4', width=900, height=253, slowmoFactor = 1)




In [ ]:
#@title STEP5: Download the Ouput
from google.colab import files
file = 'myoutput_slow.mp4' #@param ["myoutput.mp4", "myoutput_slow.mp4", "myoutput_4_display.mp4"]
filepath = {"myoutput.mp4": "/content/arXiv2020-RIFE/myoutput.mp4",
            "myoutput_slow.mp4": "/content/arXiv2020-RIFE/myoutput_slow.mp4", 
            "myoutput_4_display.mp4": "/content/arXiv2020-RIFE/myoutput_4_display.mp4"}
files.download(filepath[file])

In [ ]:
!nvidia-smi